In [2]:
# import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.preprocessing import StandardScaler

# Data Preparation

In [3]:
class CustomDataset(Dataset):
    
    def __init__(self, filepath) -> None:
        
        # load csv data
        data = pd.read_csv(filepath, header=None)
        X = data.iloc[:, :-1].values
        y = data.iloc[:, -1].values
        
        # feature scaling
        sc = StandardScaler()
        X = sc.fit_transform(X)
        
        # convert to tensors
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [4]:
dataset = CustomDataset('train_all_0.csv')

# create data indices for train val split
data_size = len(dataset)
indices = list(range(data_size))
split = int(np.floor(0.2 * data_size))
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# create data loader
train_loader = DataLoader(dataset, batch_size=16, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=16, sampler=val_sampler)

In [5]:
# create training pipeline
def train(train_loader=train_loader, val_loader=val_loader, model=None, epochs=None, criterion=None, optimizer=None):
    for epoch in range(epochs):
        
        model.train()
        train_loss = []
        train_accs = []
        
        for batch in train_loader:
            
            x, y = batch
            
            logits = model(x.to(device))
            loss = criterion(logits, y.to(device))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            acc = (logits.argmax(dim=-1) == y.to(device)).float().mean()
            train_loss.append(loss.item())
            train_accs.append(acc)
        
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)
        
        model.eval()

        valid_loss = []
        valid_accs = []
        
        for batch in val_loader:
            imgs, labels = batch
            
            with torch.no_grad():
                logits = model(imgs.to(device))
                
                acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
                
                valid_loss.append(loss.item())
                valid_accs.append(acc)
        
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)
        
        print(f'[ {epoch+1}/{epochs} ] | train_loss = {train_loss:.5f}, train_acc = {train_acc:.5f}, val_loss = {valid_loss:.5f}, val_acc = {valid_acc:.5f}')

# KNN

In [30]:
# Not necessary to use pytorch

class KNN:
    def __init__(self, train_loader, val_loader, k, weight='uniform'):
        
        # initialize data
        self.X_train = train_loader.dataset.X
        self.y_train = train_loader.dataset.y
        
        self.X_val = val_loader.dataset.X
        self.y_val = val_loader.dataset.y
        
        self.n_neighbors = k
        self.weight = weight
        
        self.n_classes = len(set((train_loader.dataset.y).numpy()))
    
    def euclidean_dist(self, a, b):
        return torch.sqrt(torch.sum((a - b)**2, dim=0))
    
    def k_neighbors(self, return_dist=False):
        
        dist = []
        neigh_ind = []
        
        point_dist = [self.euclidean_dist(x_val, self.X_train) for x_val in self.X_val]
        
        for row in point_dist:
            enum_neigh = enumerate(row)
            sorted_neigh = sorted(enum_neigh,
                                  key=lambda x: x[1])[:self.n_neighbors]
            
            ind_list = [tup[0] for tup in sorted_neigh]
            dist_list = [tup[1] for tup in sorted_neigh]
            
            dist.append(dist_list)
            neigh_ind.append(ind_list)
            
        if return_dist:
            return np.array(dist), np.array(neigh_ind)
        
        return np.array(neigh_ind)
    
    def predict(self):
        
        if self.weight == 'uniform':
            neighbors = self.k_neighbors(False)
            y_pred = np.array([
                np.argmax(np.bincount(self.y_train[neighbor]))
                for neighbor in neighbors
            ])
            
            return y_pred
        
        if self.weight == 'distance':
            dist, neigh_ind = self.k_neighbors(True)
            
            inv_dist = 1 / dist
            mean_inv_dist = inv_dist / np.sum(inv_dist, axis=1)[:, np.newaxis]
            
            prob = []
            
            for i, row in enumerate(mean_inv_dist):
                row_pred = self.y_train[neigh_ind[i]]
                
                for k in range(self.n_classes):
                    indices = np.where(row_pred == k)
                    prob_ind = np.sum(row[indices])
                    prob.append(np.array(prob_ind))
            
            predict_prob = np.array(prob).reshape(self.X_val.shape[0], self.n_classes)
            y_pred = np.array([np.argmax(item) for item in predict_prob])
            
            return y_pred
    
    def score(self):
        y_pred = self.predict()
        return float(sum(y_pred == np.array(self.y_val))) / float(len(self.y_val))

In [45]:
knn_classifier = KNN(train_loader=train_loader, val_loader=val_loader, k=2, weight='distance')
accuracy = knn_classifier.score()
print(accuracy)

0.3958333333333333


# SVM

In [15]:
def hinge_loss(output, label):
    num_labels = len(label)
    corrects = output[range(num_labels), label].unsqueeze(0).T
    
    margin = 1.0
    margins = output - corrects + margin
    loss = torch.sum(torch.max(margins, 1)[0]) / num_labels
    
    return loss

In [24]:
epochs = 10
device = torch.device('cpu')
model = nn.Linear(12, 2).to(device)
criterion = hinge_loss
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
train(train_loader, val_loader, model, epochs, criterion, optimizer)

[ 1/10 ] | train_loss = 1.34393, train_acc = 0.55208, val_loss = 1.27071, val_acc = 0.57292
[ 2/10 ] | train_loss = 1.25896, train_acc = 0.57812, val_loss = 1.19379, val_acc = 0.60417
[ 3/10 ] | train_loss = 1.19855, train_acc = 0.59635, val_loss = 1.26833, val_acc = 0.61458
[ 4/10 ] | train_loss = 1.15551, train_acc = 0.59635, val_loss = 1.09382, val_acc = 0.61458
[ 5/10 ] | train_loss = 1.12406, train_acc = 0.63021, val_loss = 1.08071, val_acc = 0.62500
[ 6/10 ] | train_loss = 1.10359, train_acc = 0.65625, val_loss = 1.07573, val_acc = 0.65625
[ 7/10 ] | train_loss = 1.08545, train_acc = 0.67448, val_loss = 1.02714, val_acc = 0.68750
[ 8/10 ] | train_loss = 1.06998, train_acc = 0.69792, val_loss = 1.05933, val_acc = 0.70833
[ 9/10 ] | train_loss = 1.05790, train_acc = 0.73698, val_loss = 1.06002, val_acc = 0.77083
[ 10/10 ] | train_loss = 1.04782, train_acc = 0.77604, val_loss = 1.10647, val_acc = 0.77083


# Softmax Classifier

In [6]:
class SoftmaxClassifier(nn.Module):
    def __init__(self):
        super(SoftmaxClassifier, self).__init__()
        self.layer_1 = nn.Linear(12, 2) 
        
    def forward(self, inputs):
        x = self.layer_1(inputs)
        
        return x

In [27]:
epochs = 10
device = torch.device('cpu')
model = SoftmaxClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
train(train_loader, val_loader, model, epochs, criterion, optimizer)

[ 1/10 ] | train_loss = 0.55169, train_acc = 0.81771, val_loss = 0.48633, val_acc = 0.85417
[ 2/10 ] | train_loss = 0.53065, train_acc = 0.83594, val_loss = 0.52339, val_acc = 0.86458
[ 3/10 ] | train_loss = 0.51362, train_acc = 0.84115, val_loss = 0.57234, val_acc = 0.86458
[ 4/10 ] | train_loss = 0.49812, train_acc = 0.84896, val_loss = 0.41153, val_acc = 0.86458
[ 5/10 ] | train_loss = 0.48502, train_acc = 0.84115, val_loss = 0.42347, val_acc = 0.87500
[ 6/10 ] | train_loss = 0.47260, train_acc = 0.84896, val_loss = 0.44053, val_acc = 0.87500
[ 7/10 ] | train_loss = 0.46226, train_acc = 0.85156, val_loss = 0.47217, val_acc = 0.86458
[ 8/10 ] | train_loss = 0.45225, train_acc = 0.85417, val_loss = 0.45541, val_acc = 0.86458
[ 9/10 ] | train_loss = 0.44400, train_acc = 0.85677, val_loss = 0.46128, val_acc = 0.87500
[ 10/10 ] | train_loss = 0.43462, train_acc = 0.86458, val_loss = 0.50207, val_acc = 0.87500


# Two Layer NN

In [9]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        self.layer_1 = nn.Linear(12, 128) 
        self.layer_out = nn.Linear(128, 2) 
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.layer_out(x)
        
        return x

In [29]:
epochs = 10
device = torch.device('cpu')
model = BinaryClassification().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
train(train_loader, val_loader, model, epochs, criterion, optimizer)

[ 1/10 ] | train_loss = 0.58412, train_acc = 0.77083, val_loss = 0.50714, val_acc = 0.79167
[ 2/10 ] | train_loss = 0.44715, train_acc = 0.81250, val_loss = 0.40733, val_acc = 0.81250
[ 3/10 ] | train_loss = 0.39121, train_acc = 0.84635, val_loss = 0.19099, val_acc = 0.84375
[ 4/10 ] | train_loss = 0.35685, train_acc = 0.86458, val_loss = 0.48609, val_acc = 0.84375
[ 5/10 ] | train_loss = 0.33304, train_acc = 0.86458, val_loss = 0.44749, val_acc = 0.84375
[ 6/10 ] | train_loss = 0.31860, train_acc = 0.87500, val_loss = 0.42144, val_acc = 0.86458
[ 7/10 ] | train_loss = 0.30729, train_acc = 0.88802, val_loss = 0.18491, val_acc = 0.86458
[ 8/10 ] | train_loss = 0.29652, train_acc = 0.88802, val_loss = 0.21103, val_acc = 0.87500
[ 9/10 ] | train_loss = 0.28481, train_acc = 0.89844, val_loss = 0.54152, val_acc = 0.86458
[ 10/10 ] | train_loss = 0.27667, train_acc = 0.90104, val_loss = 0.51886, val_acc = 0.87500
